In [14]:
import cv2
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
import os
from ipywidgets import interact
import numpy as np
import pandas as pd

# 1. 테스트 데이터셋 생성(yolov5_추론폴더)

In [3]:
label_file_path = './crack_test_dataset/labels/'
image_file_path = './crack_test_dataset/'

In [4]:
# 라벨링 txt 파일리스트 생성
label_files = [fn for fn in os.listdir(label_file_path) if fn.endswith('txt')]
print(len(label_files))
print(label_files[0])

6
apt_crack_1.txt


In [5]:
# 크랙탐지된 image 파일리스트 생성
image_files = [fn for fn in os.listdir(image_file_path) if fn.endswith('jpg')]
image_file_names = [i.split('.')[0] for i in label_files]
print(image_files)
print(image_file_names)

['apt_crack_1.jpg', 'apt_crack_10.jpg', 'apt_crack_11.jpg', 'apt_crack_12.jpg', 'apt_crack_13.jpg', 'apt_crack_14.jpg', 'apt_crack_2.jpg', 'apt_crack_3.jpg', 'apt_crack_4.jpg', 'apt_crack_5.jpg', 'apt_crack_6.jpg', 'apt_crack_7.jpg', 'apt_crack_8.jpg', 'apt_crack_9.jpg']
['apt_crack_1', 'apt_crack_11', 'apt_crack_13', 'apt_crack_6', 'apt_crack_7', 'apt_crack_9']


In [6]:
image_file_names = [i for i in image_files if i.split('.')[0] in image_file_names]
image_file_names

['apt_crack_1.jpg',
 'apt_crack_11.jpg',
 'apt_crack_13.jpg',
 'apt_crack_6.jpg',
 'apt_crack_7.jpg',
 'apt_crack_9.jpg']

In [7]:
label_file_list = [os.path.join(label_file_path,i) for i in label_files]
image_file_list = [os.path.join(image_file_path,i) for i in image_file_names]
print(label_file_list[0])
print(image_file_list[0])

./crack_test_dataset/labels/apt_crack_1.txt
./crack_test_dataset/apt_crack_1.jpg


# 2. 인덱스별 크랙정보 확인

In [27]:
# 인덱스별로 이미지 확인 함수
def show_sample(label_file_list, image_file_list, index):
    image = cv2.imread(image_file_list[index])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_bbox = image.copy()
    # 이미지의 너비와 높이
    image_width, image_height = image.shape[1], image.shape[0]
     # bbox좌표
    with open(label_file_list[index], 'r', encoding='utf-8') as f:
        # 한 줄씩 읽기
        for line in f:
            bbox_info = line.strip().split(' ')
            # 클래스 구분
            class_index = int(bbox_info[0])
            # 바운딩박스 좌표
            x, y, w, h = map(float, bbox_info[1:])
            x *= image_width
            y *= image_height
            w *= image_width
            h *= image_height
            # 정수형 좌표 계산 및 조정
            x1 = int(x - w / 2)
            y1 = int(y - h / 2)
            x2 = int(x + w / 2)
            y2 = int(y + h / 2)
            # 올바른 타입으로 사각형 그리기
            cv2.rectangle(image_bbox, (x1, y1), (x2, y2), (255, 0, 0), 3)
            # ROI
            x = int(x - (w/2))
            y = int(y - (h/2))
            w = int(w)
            h = int(h)
    image_roi = image[y:y+h, x:x+w]
    roi_gray = cv2.cvtColor(image_roi, cv2.COLOR_RGB2GRAY)
    th, roi_gray = cv2.threshold(roi_gray, 0,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    binary_image_black = roi_gray <= 10
    # x,y축별로 최대 두께 탐지
    max_crack_width_black = np.max(np.sum(binary_image_black, axis=1))
    max_crack_height_black = np.max(np.sum(binary_image_black, axis=0))
    
    plt.figure(figsize=(6,6))
    plt.title(f'탐지된 크랙 두께_가로:{max_crack_width_black}px,세로:{max_crack_height_black}px')
    plt.imshow(image_bbox)
    plt.show()

# 인터랙티브 함수 정의
@interact(index=(0, len(label_file_list)-1))
def interact_image(index):
    show_sample(label_file_list, image_file_list, index)

interactive(children=(IntSlider(value=2, description='index', max=5), Output()), _dom_classes=('widget-interac…

# 3. 데이터셋 탐지 결과 보고서 추출

In [28]:
result = []

for label, img in zip(label_file_list, image_file_list):
    image = cv2.imread(img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # 이미지의 너비와 높이
    image_width, image_height = image.shape[1], image.shape[0]
     # bbox좌표
    with open(label, 'r', encoding='utf-8') as f:
        # 한 줄씩 읽기
        for line in f:
            bbox_info = line.strip().split(' ')
            # 클래스 구분
            class_index = int(bbox_info[0])
            # 바운딩박스 좌표
            x, y, w, h = map(float, bbox_info[1:])
            x *= image_width
            y *= image_height
            w *= image_width
            h *= image_height
            # 정수형 좌표 계산 및 조정
            x1 = int(x - w / 2)
            y1 = int(y - h / 2)
            x2 = int(x + w / 2)
            y2 = int(y + h / 2)
            # ROI
            x = int(x - (w/2))
            y = int(y - (h/2))
            w = int(w)
            h = int(h)
    image_roi = image[y:y+h, x:x+w]
    roi_gray = cv2.cvtColor(image_roi, cv2.COLOR_RGB2GRAY)
    th, roi_gray = cv2.threshold(roi_gray, 0,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    binary_image_black = roi_gray <= 10
    # x,y축별로 최대 두께 탐지
    max_crack_width_black = np.max(np.sum(binary_image_black, axis=1))
    max_crack_height_black = np.max(np.sum(binary_image_black, axis=0))

    title = label.split('/')[-1].split('.')[0]
    result.append({'name':title, 'max_width':max_crack_width_black, 'max_height':max_crack_height_black})

In [29]:
crack_df = pd.DataFrame(result)
crack_df.to_excel(label_file_path+'crack-detection-result.xlsx', index=False)